In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
string_prod_pivot = pd.read_csv('project/string_prod_pivot.csv', index_col=0)
custom = pd.read_csv('project/04.Custom.csv', index_col=0)
prod = pd.read_csv('project/new_product.csv', index_col=1).drop('Unnamed: 0', axis=1)

C:\Users\acorn\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
q1 = pd.read_csv('project/q1.csv', index_col=0)
q2 = pd.read_csv('project/q2.csv', index_col=0)
q3 = pd.read_csv('project/q3.csv', index_col=0)
q4 = pd.read_csv('project/q4.csv', index_col=0)

In [4]:
display(string_prod_pivot.head())
display(custom.head())

,strings
CLNT_ID,
8,남녀공용 오베이 ...
14,남성가죽시계 로이드 ...
25,이롬황성주 일 생식 주 쉐이커 이롬 일반두유 피테로마 때박살 세트...
32,남여 간절기 셔츠 종 택 ...
33,섬유유연제 엘레강스 다우니 섬유유연제 향기지속제드라이샴푸 종 체리 ...


,CLNT_GENDER,CLNT_AGE
CLNT_ID,,
4830726,F,30
4830874,F,40
4830975,F,30
4831275,F,30
4825325,F,30


In [ ]:
# 고객 입력
clnt = custom.loc[6592475]
clnt

CLNT_GENDER     F
CLNT_AGE       30
Name: 6592475, dtype: object

In [ ]:
# 성별,나이로 분류 후, q1/q2/q3/q4로 재분류
def classify(clnt):
    custom_gen = custom[(custom['CLNT_GENDER'] == clnt[0]) & (custom['CLNT_AGE'] == clnt[1])]
    if clnt.name in q1.index:
        return custom_gen.loc[q1.index].dropna()
    elif clnt.name in q2.index:
        return custom_gen.loc[q2.index].dropna()
    elif clnt.name in q3.index:
        return custom_gen.loc[q3.index].dropna()
    elif clnt.name in q4.index:
        return custom_gen.loc[q4.index].dropna()
    else:
        print('client_not_found')

In [ ]:
classed = classify(clnt)

C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
classed_fin = string_prod_pivot.loc[classed.index].dropna()

C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [ ]:
# 벡터, 행렬화
tf = TfidfVectorizer()
tf_matrix = tf.fit_transform(classed_fin['strings'])
print(tf_matrix.shape)

(27093, 59080)


In [ ]:
#유사도
cosine_sim = cosine_similarity(tf_matrix, tf_matrix)

In [ ]:
indices = pd.Series(np.arange(len(classed_fin)), index = classed_fin.index)

In [ ]:
# 행렬에서 값 뽑기
def similarity(clnt, cosine_sim=cosine_sim):
    idx = indices[clnt.name]

    score = list(enumerate(cosine_sim[idx]))

    score = sorted(score, key=lambda x: x[1], reverse=True)

    score = score[1:6]

    indices_re = [i[0] for i in score]

    return classed_fin.iloc[indices_re].index, score

In [ ]:
similarity(clnt)

(Int64Index([3475810, 4299816, 6241401, 1528920, 6172081], dtype='int64', name='CLNT_ID'),
 [(16866, 0.5642103422862768),
  (19517, 0.5041172979356562),
  (25713, 0.48328108198772196),
  (9124, 0.47589635157700716),
  (25454, 0.4757424433220827)])

In [ ]:
recommend = prod[['PD_C', 'PD_BRA_NM', 'PD_BUY_CT']].loc[similarity(clnt)[0]]
recommend = recommend.groupby(['PD_C','PD_BRA_NM'])[['PD_BUY_CT']].sum().sort_values('PD_BUY_CT', ascending=False)

In [ ]:
recommend.reset_index()[['PD_C','PD_BRA_NM']][0:5]

In [ ]:
prod[prod.index==6592475]